In [ ]:
import numpy
import xarray
import matplotlib.pyplot as mp
import glob
import pandas
import cartopy
import itertools
import datetime
import calendar

import dask_jobqueue
import dask.distributed
import ipywidgets
#import dask.diagnostics

useful commands

In [ ]:
cluster = dask_jobqueue.PBSCluster(cores=36,
                                   memory='100 GB',
                                   project='UCLA0022',
                                   queue='regular',
                                   walltime='01:00:00',
                                   local_directory='$TMPDIR')

In [ ]:
# ask for 30 workers with these specifications
cluster.start_workers(50)

In [ ]:
print(cluster.job_script())

In [ ]:
client = dask.distributed.Client(cluster)
client

In [ ]:
#PRECT_root = '/ninad/NCAR_LENS/PRECT/'
#PRECT_root = '/scratch/blangenb/NCAR_LENS/PRECT/'
PRECT_root =  '/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/daily/PRECT/'

Open pre-industrial control as multi-file dataset

In [ ]:
filename = 'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.04020101-04991231.nc'
example_pic_ds = xarray.open_dataset(PRECT_root + 'b.e11.B1850C5CN.f09_g16.005.cam.h1.PRECT.04020101-04991231.nc')
lat_total = example_pic_ds['lat']
lon_total = example_pic_ds['lon']

In [ ]:
filename_list = sorted(glob.glob('/glade/scratch/baird/global-whiplash-files/rolling_30day_sum*'))

#mfds = xarray.open_mfdataset([filename_list[0]], chunks={'lat':24, 'lon':24})
#mfds = xarray.open_mfdataset(filename_list[:10], chunks={'lat':24, 'lon':24})
mfds = xarray.open_mfdataset(filename_list, chunks={'lat':24, 'lon':24})

mfds_data = mfds['__xarray_dataarray_variable__']

mfds_time = mfds['time'].values
mfds_lat = mfds['lat'].values
mfds_lon = mfds['lon'].values

In [ ]:
mfds_lat.shape, mfds_lon.shape

1. pull out time and specific months
2. calculate the number of days in those months
3. other stuff

In [ ]:
event_size = 30 # days
season = 'ONDJFM'; month_numbers = [10,11,12,1,2,3]
time_data_indices = [t.month in month_numbers for t in mfds_time]

ndays = sum([calendar.monthrange(1979,m)[1] for m in month_numbers])

In [ ]:
return_period = 100 # in years
events_per_year = ndays/event_size #152/30

return_val_perc = 100*(1-1/(return_period*events_per_year))
return_val_fraction = (1-1/(return_period*events_per_year))
print(return_val_perc)

In [ ]:
print('mfds size in GB {:0.2f}\n'.format(mfds.nbytes / 1e9))
mfds.info()

WORKED ON SMALL DATA

TRYING FOR LARGER DATA SET

In [ ]:
mfds_data

In [ ]:
mfds_data = mfds_data.persist()

In [ ]:
def calc_perc(x, q=return_val_fraction):
    return(numpy.nanpercentile(x, q=return_val_fraction, axis=0))

doesn't compute things yet, just applies function to all chunks:

In [ ]:
result = dask.array.apply_along_axis(calc_perc, arr=mfds_data.data, axis=0)

now do the computation

In [ ]:
perc_calculation = result.compute()

In [ ]:
mp.pcolormesh(perc_calculation*86400.*1000.)
mp.colorbar()

In [ ]:
output_da = xarray.DataArray(perc_calculation*86400.*1000., name='100yr_30dsum_return_values', 
                             coords={'lon':lon_total,'lat':lat_total},
                             dims={'lat','lon'}, attrs={'units':'kg m s-1'})

output_df = xarray.Dataset({'100yr_30dsum_return_values':output_da})

In [ ]:
output_da

In [ ]:
output_df

In [ ]:
output_df.to_netcdf('pic_100yr_30dsum_return_values.nc')

In [ ]:
!ls *.nc